<a href="https://colab.research.google.com/github/voidnire/DeepLearning/blob/cursos/SarcasmsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet
!pip install opendatasets --quiet

import opendatasets as od
od.download("https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: erindante
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection


100%|██████████| 3.30M/3.30M [00:00<00:00, 524MB/s]

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Available Device is ", device)

Available Device is  cpu


In [5]:
data_df = pd.read_json("/content/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json", lines=True)
data_df.dropna(inplace = True)
data_df.drop_duplicates(inplace = True)

print(data_df.shape)
data_df.head()

(26708, 3)


,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [6]:
SEED = 23

In [11]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data_df['headline']),
                                                    np.array(data_df['is_sarcastic']),
                                                    test_size=0.2, random_state=SEED)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

print("Training Size: ", X_train.shape[0], "- ",round(X_train.shape[0]/data_df.shape[0],4)*100,"%")
print("Validation Size: ", X_val.shape[0], "- ",round(X_val.shape[0]/data_df.shape[0],4)*100,"%")
print("Testing Size: ", X_test.shape[0], "- ",round(X_test.shape[0]/data_df.shape[0],4)*100,"%")


Training Size:  21366 -  80.0 %
Validation Size:  2671 -  10.0 %
Testing Size:  2671 -  10.0 %


In [12]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
bert_model = AutoModel.from_pretrained('google-bert/bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [17]:
class dataset(Dataset):
  def __init__(self, X, Y):
    self.X = [tokenizer(x,
                        max_length = 100,
                        truncation = True,
                        padding = "max_length", # [1,3,2,0,0,0,0,0,..,0]
                        return_tensors = "pt" #pytorch tensors
                        ).to(device) for x in X
    ]
    self.Y = torch.tensor(Y, dtype = torch.float32).to(device)
    #self.tokenizer = tokenizer

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


In [18]:
training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

- Video:
https://www.youtube.com/watch?v=E0bwEAWmVEM

HYPERPARAMS

In [19]:
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4

In [ ]:
train_dataloader = DataLoader()